In [73]:
import pandas as pd
import numpy as np

import time

from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [ ]:
RANDOM_STATE = 42

In [ ]:
raw_df = pd.read_csv('./data/boston.csv')
raw_df.head()

In [ ]:
X = raw_df.iloc[:,0:14]
y = raw_df.iloc[:,13:14]

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [ ]:
regressions = [ LinearRegression(),
               Lasso(),
               Ridge()
               ]

for regression in regressions:
    start_time = time.time()
    pipe = Pipeline(steps=[('regression', regression)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    elapsed_time = time.time() - start_time
    print('Модель: ',regression)
    print('Время выполнения: %.3f' %elapsed_time)
    print('Коэффициент регрессии',round(r2_score(y_test, y_pred),10))
    print('----')

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [ ]:
coef_reg = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]


In [ ]:
regressions = [ Lasso,
                Ridge
               ]

In [ ]:
for regression in regressions:

    params = [{'alpha': coef_reg}]
    search = GridSearchCV(regression(), params, scoring='r2',cv=5).fit(X_train, y_train)
    r = regression(search.best_params_['alpha']).fit(X_train, y_train)
    y_pred = r.predict(X_test)

    print('Модель: ',regression())
    print('Коэффициент регуляризации: ', search.best_params_['alpha'])
    print('Коэффициент регрессии: ',round(r2_score(y_test, y_pred),10))
    print('----')
    



In [ ]:
regressions_CV = [ LassoCV,
                RidgeCV
               ]

for regression in regressions_CV:

    params = [{'alphas': coef_reg}]
    search = GridSearchCV(regression(), params, scoring='r2',cv=5).fit(X_train, y_train)
    r = regression(search.best_params_['alpha']).fit(X_train, y_train)
    y_pred = r.predict(X_test)

    print('Модель: ',regression())
    print('Коэффициент регуляризации: ', search.best_params_['alpha'])
    print('Коэффициент регрессии: ',round(r2_score(y_test, y_pred),10))
    print('----')

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [ ]:
lasso_pipe = Pipeline(
                        [
                            ('mms', MinMaxScaler()),
                            ('ss', StandardScaler()),
                            ('lasso', Lasso())
                        ]
                    )

In [ ]:
coef_reg = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]

params = [{'lasso__alpha': coef_reg }]

gs_lasso = GridSearchCV(
                        lasso_pipe,
                        param_grid=params,
                        scoring='r2',
                        cv=5,
                        n_jobs=-1,
                        verbose=2
)

In [ ]:
model = gs_lasso.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print('Коэффициент регрессии: ',round(r2_score(y_test, y_pred),10))

In [ ]:
scaler = MinMaxScaler()
pipe = scaler.fit(X_train, y_train)
# pipe.transform(X_train, y_train)

In [ ]:
pipe.get_params()

In [ ]:
transformer = Pipeline(
                       [
                           ('MinMax', MinMaxScaler()),
                          
                       ] 
                      )

In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
numeric_features

In [ ]:
preprocessor = ColumnTransformer(transformers=[
                                 'scaler', transformer, numeric_features
                                ])                           

In [ ]:
pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('regression', Lasso())
    ]
)

In [ ]:
model = pipe.fit_transform(X_train, y_train)

In [ ]:
regressions = [ Lasso,
                Ridge
               ]

for regression in regressions:
    start_time = time.time()
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed_time = time.time() - start_time
    print('Модель: ',regression)
    print('Время выполнения: %.3f' %elapsed_time)
    print('Коэффициент регрессии',round(r2_score(y_test, y_pred),10))
    print('----')

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [ ]:
data.head()

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.